# Engagement RNN Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics

# 1 -- Load, Choose Dataset

### Choose Datasets: Regular, Smoothed, Windowed

In [2]:
# Regular Data: All Frames, No Smoothing 
# p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p5_master.csv'
# p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p7_master.csv'
# p8_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p8_master.csv'
# p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/p9_master.csv'

# Smoothed Data: All Frames, Smoothing of Open Face Features 
#p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p5_master_smooth.csv'
#p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p7_master_smooth.csv'
#p8_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p8_master_smooth.csv'
#p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Smooth/p9_master_smooth.csv'

# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p7_master_window (1).csv'
p8_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p8_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master_Window/p9_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data8 = pd.read_csv(p8_file)
data9 = pd.read_csv(p9_file)

In [3]:
data = [data5, data7, data8, data9]
part = [5, 7, 8, 9]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [4]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'p_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
engagement_change
engagement_var
participant
session_num
timestamp


In [5]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if 'of_' in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU01_change
of_AU02_c
of_AU02_change
of_AU04_c
of_AU04_change
of_AU05_c
of_AU05_change
of_AU06_c
of_AU06_change
of_AU07_c
of_AU07_change
of_AU09_c
of_AU09_change
of_AU10_c
of_AU10_change
of_AU12_c
of_AU12_change
of_AU14_c
of_AU14_change
of_AU15_c
of_AU15_change
of_AU17_c
of_AU17_change
of_AU20_c
of_AU20_change
of_AU23_c
of_AU23_change
of_AU25_c
of_AU25_change
of_AU26_c
of_AU26_change
of_AU28_c
of_AU28_change
of_AU45_c
of_AU45_change
of_confidence
of_confidence_var
of_gaze_0_x
of_gaze_0_x_var
of_gaze_0_y
of_gaze_0_y_var
of_gaze_0_z
of_gaze_0_z_var
of_gaze_1_x
of_gaze_1_x_var
of_gaze_1_y
of_gaze_1_y_var
of_gaze_1_z
of_gaze_1_z_var
of_gaze_angle_x
of_gaze_angle_x_var
of_gaze_angle_y
of_gaze_angle_y_var
of_gaze_distance
of_gaze_distance_var
of_pose_Rx
of_pose_Rx_var
of_pose_Ry
of_pose_Ry_var
of_pose_Rz
of_pose_Rz_var
of_pose_Tx
of_pose_Tx_var
of_pose_Ty
of_pose_Ty_var
of_pose_Tz
of_pose_Tz_var
of_pose_distance
of_pose_distance_var
of_success
of_success_change
of_ts_success


In [6]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_harmonicity_var
a_intensity
a_intensity_var
a_mfcc_0
a_mfcc_0_var
a_mfcc_1
a_mfcc_1_var
a_pitch_frequency
a_pitch_frequency_var
a_pitch_strength
a_pitch_strength_var


In [7]:
# Performance Columns

p_cols = []
for i in data[0].columns:
    if 'p_' in i and 'op_' not in i:
        p_cols.append(i)
        
p_cols = sorted(p_cols)
for i in p_cols:
    print(i)

p_aptitude
p_aptitude_var
p_diff_1
p_diff_2
p_diff_3
p_diff_4
p_diff_5
p_games_change
p_games_session
p_games_total
p_mistakes_change
p_mistakes_game
p_mistakes_session
p_mistakes_total
p_no_game
p_playing_change
p_skill_EM
p_skill_NC
p_skill_OS
p_ts_attempt
p_ts_game_played
p_ts_game_start
p_ts_robot_talked


In [8]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_change
of_AU02_change
of_AU04_change
of_AU05_change
of_AU06_change
of_AU07_change
of_AU09_change
of_AU10_change
of_AU12_change
of_AU14_change
of_AU15_change
of_AU17_change
of_AU20_change
of_AU23_change
of_AU25_change
of_AU26_change
of_AU28_change
of_AU45_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
op_num_people_var
p_aptitude_var
p_games_change
p_mistakes_change
p_playing_change


### Filter Feature Set (Optional)

In [9]:
# features_to_keep = data[0].columns

# always include basic_cols, add desired group of features 
# features_to_keep = basic_cols + of_cols + p_cols + a_cols

# for i,d in enumerate(data):
#    data[i] = d[features_to_keep]

# 3 -- Train-Test Split

In [10]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_train, split_test

In [11]:
# Function: Formulate Train-Test Split 

# train_part: an array of participants 
# test_part: single participant
# split_size: how much of test_part data to use for testing
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i])
    
    i = part.index(test_part)
    test_data = data[i]
    if split_size < 1:
        split_train, test_data = part_split(test_data, split_size)
        train_data.append(split_train)
    
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    
    return train_data, test_data

In [12]:
# Train-Test Split

# Excluding Participant 8
perc = 1
train, test = split(train_part=[5,7], test_part=9, split_size=perc)
#train, test = split(train_part=[5,9], test_part=7, split_size=perc)
# train, test = split(train_part=[7,9], test_part=5, split_size=perc)

# Individualized
# perc = 0.7
# train, test = split(train_part=[], test_part=7, split_size=perc)
# train, test = split(train_part=[], test_part=5, split_size=perc)
# train, test = split(train_part=[], test_part=9, split_size=perc)
# train, test = split(train_part=[], test_part=8, split_size=perc)

# 4 -- Scenario Based Modeling

- During Game / Outside of Game
- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [ ]:
# Create Separate Models for Different Scenarios

# # Outside of Game
# train = train.loc[train['no_game']==1]
# test = test.loc[test['no_game']==1]

# # During Game
# train = train.loc[train['no_game']==0]
# test = test.loc[test['no_game']==0]

# # Open Face Success
# train = train.loc[train['of_success']==1]
# test = test.loc[test['of_success']==1]

# # Open Face Failure
# train = train.loc[train['of_success']==0]
# test = test.loc[test['of_success']==0]

# # Robot Talking
# train = train.loc[train['p_ts_robot_talked']==0]
# test = test.loc[test['p_ts_robot_talked']==0]

# # Robot Not Talking
# train = train.loc[train['p_ts_robot_talked']>0]
# test = test.loc[test['p_ts_robot_talked']>0]

# # First 10 Minutes of Session
# train = train.loc[train['timestamp']<=(10*60)]
# test = test.loc[test['timestamp']<=(10*60)]

# # After first 10 Minutes of Session
# train = train.loc[train['timestamp']>(10*60)]
# test = test.loc[test['timestamp']>(10*60)]

### Divide into X_train, y_train, X_test, y_test

In [13]:
train.shape

(40573, 116)

In [14]:
test.shape

(24419, 116)

In [ ]:
train.columns

In [15]:
# make sure data is sorted by session num
train = train.sort_values(by=['participant', 'session_num'])
test = test.sort_values(by=['participant', 'session_num'])

### Create windows for RNN

In [16]:
window_size = 60
windows = []
X_train = []
y_train = []
X_test = []
y_test = []

In [17]:
curr_session = -1
window_start = -1
curr_window = -1
for i,r in train.iterrows():
    this_session = r['session_num']
    #print(this_session)
    
    # ignore remainder of window if encounter new session
    if (int(curr_session) != (this_session)):
        curr_session = this_session
        window_start = i
        curr_window = 1
        continue
        
     # process the window if it's the same session
    if (int(curr_window) == int(window_size - 1)):
        to_add = train[window_start:i+1]
        to_add = to_add.drop(columns=['session_num', 'timestamp'])
        
        windows.append(to_add)
        y_train.append(to_add['engagement'])
        X_train.append(to_add.drop(columns=['engagement']))
    
        window_start = i+1
        curr_window = 0
        continue 
        
    curr_window += 1

In [18]:
y_train[:1]

[0     1.0
 1     1.0
 2     1.0
 3     1.0
 4     1.0
 5     1.0
 6     1.0
 7     1.0
 8     1.0
 9     1.0
 10    1.0
 11    1.0
 12    1.0
 13    1.0
 14    1.0
 15    1.0
 16    1.0
 17    1.0
 18    1.0
 19    1.0
 20    1.0
 21    1.0
 22    1.0
 23    1.0
 24    1.0
 25    1.0
 26    1.0
 27    1.0
 28    1.0
 29    1.0
 30    1.0
 31    1.0
 32    1.0
 33    1.0
 34    1.0
 35    1.0
 36    1.0
 37    1.0
 38    1.0
 39    1.0
 40    1.0
 41    1.0
 42    1.0
 43    1.0
 44    1.0
 45    1.0
 46    1.0
 47    1.0
 48    1.0
 49    1.0
 50    1.0
 51    1.0
 52    1.0
 53    1.0
 54    1.0
 55    1.0
 56    1.0
 57    1.0
 58    1.0
 59    1.0
 Name: engagement, dtype: float64]

In [19]:
curr_session = -1
window_start = -1
curr_window = -1
for i,r in test.iterrows():
    this_session = r['session_num']
    
    # we've encountered a new session
    if curr_session != this_session:
        # ignore remainder window
        curr_session = this_session
        window_start = i
        curr_window = 1
        continue
        
     # we've found a complete window
    if curr_window == window_size - 1:
        to_add = train[window_start:i+1]
        to_add = to_add.drop(columns=['session_num', 'timestamp'])
        
        windows.append(to_add)
        y_test.append(to_add['engagement'])
        X_test.append(to_add.drop(columns=['engagement']))
    
        window_start = i+1
        curr_window = 0
        continue 
        
    curr_window += 1


In [20]:
X_test[:1]

[    a_harmonicity  a_harmonicity_var  a_intensity  a_intensity_var   a_mfcc_0  \
 0          -200.0       10758.017444      64.9555        33.398698  1012.5155   
 1          -200.0        4342.190562      62.5270        29.359142   950.8675   
 2          -200.0       11399.013125      64.5445        17.898118   871.8310   
 3          -200.0        9921.207787      62.6840        66.748154   841.0335   
 4          -200.0        6861.886618      53.0585        52.916818   856.1160   
 5          -200.0        6861.886618      53.4415        47.317505   777.2935   
 6          -200.0        7052.599109      56.6705        52.865598   923.6635   
 7          -200.0        7052.599109      63.0545        18.297550   954.4280   
 8          -200.0           0.000000      57.1965       444.893232   855.9585   
 9          -200.0        4121.832602      50.9240       418.227762   702.8555   
 10         -200.0        4121.832602      59.1255        52.112485   787.8115   
 11         -200

In [ ]:
# Do NOT Sort Training Data Together (RNN is a sequential model) 

# X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
# X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
# X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
# X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
# X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

# 5 -- Preprocessing
Standardization Recommended

### Min-Max Scaling
- x' = ( x - min(x) ) / ( max(x) - min(x) )
- => x' in [0, 1], fill NaN with -1

In [ ]:
# for c in X_train.columns:
#     min_val = np.nanmin(X_train[c])
#     max_val = np.nanmax(X_train[c])
    
#     X_train[c] = (X_train[c]-min_val)/(max_val - min_val)
#     X_test[c] = (X_test[c]-min_val)/(max_val - min_val)
    
# X_train = X_train.fillna(-1)
# X_test = X_test.fillna(-1)

### Standardization (Recommended)
- x' = ( x - mean(x) ) / ( stdev(x) )
- => x' is z-score, NaN's filled with min_val

In [21]:
X_train[0]

,a_harmonicity,a_harmonicity_var,a_intensity,a_intensity_var,a_mfcc_0,a_mfcc_0_var,a_mfcc_1,a_mfcc_1_var,a_pitch_frequency,a_pitch_frequency_var,...,p_no_game,p_playing_change,p_skill_EM,p_skill_NC,p_skill_OS,p_ts_attempt,p_ts_game_played,p_ts_game_start,p_ts_robot_talked,participant
0,-200.0,10758.017444,64.9555,33.398698,1012.5155,8551.985609,452.1725,5859.157649,540.2025,3.284413e+06,...,0.0,0,0.0,0.0,1.0,0.483334,0.0,0.483334,0.000000,5.0
1,-200.0,4342.190562,62.5270,29.359142,950.8675,8985.921989,493.4385,4044.100149,341.1745,3.573045e+05,...,0.0,0,0.0,0.0,1.0,0.983333,0.0,0.983333,0.000000,5.0
2,-200.0,11399.013125,64.5445,17.898118,871.8310,8779.073602,513.1285,2761.766612,0.0000,5.634695e+04,...,0.0,0,0.0,0.0,1.0,1.483333,0.0,1.483333,0.000000,5.0
3,-200.0,9921.207787,62.6840,66.748154,841.0335,8804.661948,475.9850,9116.637209,545.0580,6.202114e+06,...,0.0,0,0.0,0.0,1.0,1.983334,0.0,1.983334,0.000000,5.0
4,-200.0,6861.886618,53.0585,52.916818,856.1160,19833.670992,421.0085,7781.401123,938.7650,5.782847e+06,...,0.0,0,0.0,0.0,1.0,2.483334,0.0,2.483334,0.000000,5.0
5,-200.0,6861.886618,53.4415,47.317505,777.2935,21572.488161,411.9710,8423.693107,312.3700,3.819848e+06,...,0.0,0,0.0,0.0,1.0,2.983334,0.0,2.983334,0.000000,5.0
6,-200.0,7052.599109,56.6705,52.865598,923.6635,22673.856229,412.6710,9139.360425,312.3700,9.235121e+06,...,0.0,0,0.0,0.0,1.0,3.483334,0.0,3.483334,0.000000,5.0
7,-200.0,7052.599109,63.0545,18.297550,954.4280,7863.561577,472.7955,6039.262230,655.0370,9.145406e+06,...,0.0,0,0.0,0.0,1.0,3.983334,0.0,3.983334,0.000000,5.0
8,-200.0,0.000000,57.1965,444.893232,855.9585,629762.415145,418.9480,30487.578308,1020.8145,9.174430e+06,...,0.0,0,0.0,0.0,1.0,4.483333,0.0,4.483333,0.000000,5.0
9,-200.0,4121.832602,50.9240,418.227762,702.8555,602795.049361,403.7660,30911.631520,986.1730,6.548317e+06,...,0.0,0,0.0,0.0,1.0,4.983333,0.0,4.983333,0.000000,5.0


In [ ]:
for c in X_train:
    #iterate through columns and standardize values
#     mean = np.nanmean(X_train[c])
#     std = np.nanstd(X_train[c])
    
#     X_train[c] = (X_train[c]-mean)/(std)
#     X_test[c] = (X_test[c]-mean)/(std)
        
#     if 'of_pose_distance' not in c:
#         min_val = np.nanmin(X_train[c])

#         X_train[c] = X_train[c].fillna(min_val)
#         X_test[c] = X_test[c].fillna(min_val)
#     else:
#         max_val = np.nanmax(X_train[c])

#         X_train[c] = X_train[c].fillna(max_val)
#         X_test[c] = X_test[c].fillna(max_val)

# 6 -- Model!

In [ ]:
#RNN imports
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.optimizers import RMSprop
from keras import metrics
from keras.models import Model

In [ ]:
len(X_train)

In [ ]:
rnn_model = Sequential()

# At this point, each individual training sample is now a sequence of word embedding vectors

#rnn_model.add(Dropout(.2))
rnn_model.add(LSTM(64, input_shape=(60, 113), return_sequences=False, dropout=0.2, recurrent_dropout=0.1))
# Dense 2 (final vote)
rnn_model.add(Dense(50, activation = 'sigmoid'))

print(rnn_model.summary())

# Define Loss and Optimizer
LOSS = 'binary_crossentropy' #classifying between 0 and 1
OPTIMIZER = 'rmsprop' #RMSprop typically works better for RNNs per Keras

rnn_model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64

rnn_model.fit(X_train, y_train, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =0.2)

In [ ]:
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [ ]:
loss_and_metrics = rnn_model.evaluate(X_test, y_test)
print(loss_and_metrics)

In [ ]:
scores = rnn_model.predict_proba(X_test)
pred = rnn_model.predict_classes(X_test)

# 7 -- Evaluation

In [ ]:
# y_test: actual y values for test data
# scores: predicted probabilistic y values for test data
# pred: predicted binary y values for test data

In [ ]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))